In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train = pd.read_csv("/Users/jimmywang/Desktop/Jupyter/ML/Final Report/train.csv")
print ( "Number of NaN in the Data" )
print ( train.isna().sum() )

Number of NaN in the Data
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [3]:
test = pd.read_csv("/Users/jimmywang/Desktop/Jupyter/ML/Final Report/test.csv")
print ( "Number of NaN in the Data" )
print ( test.isna().sum() )

Number of NaN in the Data
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [4]:
output_label = test['PassengerId']
train = train.drop(['PassengerId'], axis=1)
test = test.drop(['PassengerId'], axis=1)

In [5]:
train_anal = train.select_dtypes(include = ["int64", "float64" ])
correlation_matrix = train_anal.corr()
correlation_with_target = correlation_matrix['Survived']
print(correlation_with_target.abs().sort_values(ascending=False))

Survived    1.000000
Pclass      0.338481
Fare        0.257307
Parch       0.081629
Age         0.077221
SibSp       0.035322
Name: Survived, dtype: float64


In [6]:
train = train.drop(['Cabin'], axis=1)
test = test.drop(['Cabin'], axis=1)
train = train.drop(['Name'], axis=1)
test = test.drop(['Name'], axis=1)
train = train.drop(['Ticket'], axis=1)
test = test.drop(['Ticket'], axis=1)

In [7]:
from sklearn.preprocessing import LabelEncoder
columns_to_encode_train = train.columns[train.dtypes == 'object']  # 只選擇類型為'object'（字符串）的列
columns_to_encode_test = test.columns[test.dtypes == 'object']  # 只選擇類型為'object'（字符串）的列
labelencoder = LabelEncoder()
for i in columns_to_encode_train:
    train[i] = labelencoder.fit_transform(train[i])
train
for i in columns_to_encode_test:
    test[i] = labelencoder.fit_transform(test[i])
test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.5,0,0,7.8292,1
1,3,0,47.0,1,0,7.0000,2
2,2,1,62.0,0,0,9.6875,1
3,3,1,27.0,0,0,8.6625,2
4,3,0,22.0,1,1,12.2875,2
...,...,...,...,...,...,...,...
413,3,1,NaN,0,0,8.0500,2
414,1,0,39.0,0,0,108.9000,0
415,3,1,38.5,0,0,7.2500,2
416,3,1,NaN,0,0,8.0500,2


In [8]:
y_train = train["Survived"]
x_train = train.drop(["Survived"], axis=1)
x_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,2
1,1,0,38.0,1,0,71.2833,0
2,3,0,26.0,0,0,7.9250,2
3,1,0,35.0,1,0,53.1000,2
4,3,1,35.0,0,0,8.0500,2
...,...,...,...,...,...,...,...
886,2,1,27.0,0,0,13.0000,2
887,1,0,19.0,0,0,30.0000,2
888,3,0,NaN,1,2,23.4500,2
889,1,1,26.0,0,0,30.0000,0


In [9]:
x_train = x_train.interpolate(method='spline', order=3)
test = test.interpolate(method='spline', order=3)
print ( x_train.isna().sum() )
print ( test.isna().sum() )
test

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.500000,0,0,7.8292,1
1,3,0,47.000000,1,0,7.0000,2
2,2,1,62.000000,0,0,9.6875,1
3,3,1,27.000000,0,0,8.6625,2
4,3,0,22.000000,1,1,12.2875,2
...,...,...,...,...,...,...,...
413,3,1,29.447858,0,0,8.0500,2
414,1,0,39.000000,0,0,108.9000,0
415,3,1,38.500000,0,0,7.2500,2
416,3,1,9.838002,0,0,8.0500,2


In [10]:
# 正規劃處理 
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
scaler_test = StandardScaler().fit(test)
test = scaler_test.transform(test)

In [11]:
classifier = RandomForestClassifier( n_estimators = 1000, random_state = 42 )

In [12]:
classifier.fit( x_train, y_train)

RandomForestClassifier(n_estimators=1000, random_state=42)

In [13]:
pred = classifier.predict(test)

In [14]:
output = list()
for i in range(len(pred)):
    output.append([output_label[i], pred[i]])

In [15]:
output_df = pd.DataFrame( output , columns=["PassengerId", "Survived"])

In [16]:
output_df .to_csv( "/Users/jimmywang/Desktop/Jupyter/ML/Final Report/Output.csv", encoding = 'utf-8',index = False)